# Train, promote and deploy Boston house prices prediction model

This notebook contains steps and code to demonstrate support of AI Lifecycle features in Cloud Pak for Data. 
It contains steps and code to work with [`cpdctl`](https://github.com/IBM/cpdctl) CLI tool available in IBM github repository. 
It also introduces commands for getting model and training data, persisting model, deploying model
and promoting it between deployment spaces.

Some familiarity with Python is helpful. This notebook uses Python 3.7.


In [2]:
import base64
import json
import os
import platform
import requests
from IPython.core.display import display, HTML

## CPD Credentials

In [3]:
CPD35_USER_NAME = 'YOUR CPD 3.0.1 user name'
CPD35_USER_PASSWORD = 'YOUR CPD 3.0.1 user password'
CPD35_URL = 'YOUR CPD 3.0.1 CLUSTER URL'

### Install the latest version of `cpdctl`

In [83]:
PLATFORM = platform.system().lower()
CPDCTL_ARCH = "{}_amd64".format(PLATFORM)
CPDCTL_RELEASES_URL="https://api.github.com/repos/IBM/cpdctl/releases"
CWD = os.getcwd()
PATH = os.environ['PATH']
CPDCONFIG = os.path.join(CWD, '.cpdctl.config.yml')

response = requests.get(CPDCTL_RELEASES_URL)
assets = response.json()[0]['assets']
platform_asset = next(a for a in assets if CPDCTL_ARCH in a['name'])
cpdctl_url = platform_asset['url']
cpdctl_file_name = platform_asset['name']

response = requests.get(cpdctl_url, headers={'Accept': 'application/octet-stream'})
with open(cpdctl_file_name, 'wb') as f:
    f.write(response.content)
    
display(HTML('<code>cpdctl</code> binary downloaded from: <a href="{}">{}</a>'.format(platform_asset['browser_download_url'], platform_asset['name'])))

In [84]:
%%capture

%env PATH={CWD}:{PATH}
%env CPDCONFIG={CPDCONFIG}

In [85]:
if CPDCONFIG and os.path.exists(CPDCONFIG):
    os.remove(CPDCONFIG)
    
! tar -xzf cpdctl.tag.gz

version_r = ! cpdctl version
CPDCTL_VERSION = version_r.s

print("cpdctl version: {}".format(CPDCTL_VERSION))

cpdctl version: 0.4.23


# CPDCTL Demo

AI Lifecycle automation using `cpdctl` CLI tool with CPD 3.5 cluster.

### Add CPD 3.5 cluster configuration

Add "cpd35_user" user to the `cpdctl` configuration

In [8]:
! cpdctl config user set cpd35_user --username {CPD35_USER_NAME} --password {CPD35_USER_PASSWORD}

Add "cpd35" profile to the `cpdctl` configuration

In [9]:
! cpdctl config profile set cpd35 --url {CPD35_URL} --user cpd35_user

Add "cpd35" context to the `cpdctl` configuration

In [10]:
! cpdctl config context set cpd35 --profile cpd35 --user cpd35_user

List available contexts

In [11]:
! cpdctl config context list

Name    Profile   User         Current   
cpd35   cpd35     cpd35_user   *   


In [12]:
! cpdctl config context use cpd35

Switched to context "cpd35".


List available projects in current context

In [13]:
! cpdctl project list

...
ID                                     Name          Created                    Description   Tags   
7fb76cf7-25be-435d-818e-bd6e9b5254f5   cpdctl-demo   2021-01-29T08:01:23.363Z                 []   


### Access the selected project assets

Get project the first project ID and show details

In [14]:
result = ! cpdctl project list --output json --raw-output --jmes-query 'resources[0].metadata.guid'
PROJECT_ID = result.s

In [15]:
! cpdctl project get --project-id {PROJECT_ID}

...
                  
ID:            7fb76cf7-25be-435d-818e-bd6e9b5254f5   
Name:          cpdctl-demo   
Created:       2021-01-29T08:01:23.363Z   
Description:      
Tags:          []   


Get project details in JSON format and extract it's name

In [16]:
! cpdctl project get --project-id {PROJECT_ID} --output json

{
  "entity": {
    "creator": "demouser",
    "creator_iam_id": "1000331004",
    "description": "",
    "name": "cpdctl-demo",
    "public": false,
    "scope": {
      "bss_account_id": "999",
      "enforce_members": true
    },
    "storage": {
      "guid": "95590fa0-1246-499d-84a8-7cb030d14112",
      "type": "assetfiles"
    }
  },
  "metadata": {
    "created_at": "2021-01-29T08:01:23.363Z",
    "guid": "7fb76cf7-25be-435d-818e-bd6e9b5254f5",
    "updated_at": "2021-01-29T08:01:27.103Z",
    "url": "/v2/projects/7fb76cf7-25be-435d-818e-bd6e9b5254f5"
  }
}


In [17]:
result = ! cpdctl project get --project-id {PROJECT_ID} --output json --jmes-query "entity.name" --raw-output
PROJECT_NAME = result.s
print("'{}' project ID is: {}".format(PROJECT_NAME, PROJECT_ID))

'cpdctl-demo' project ID is: 7fb76cf7-25be-435d-818e-bd6e9b5254f5


List assets in the project

In [19]:
! cpdctl asset search --project-id {PROJECT_ID} --type-name asset --query "*:*"

...
ID                                     Name                       Created                    Description                 Type         State       Tags         Size   
edb6fe21-77c4-4cb3-aa6d-9e36d2b18edd   credit_risk_training.csv   2021-01-29T08:51:00.000Z                               data_asset   available   []           689622   
ceea9923-7ff7-4084-a560-818716e65b4d   Sample notebook            2021-01-29T08:51:49.000Z   Notebook run using cpdctl   notebook     available   [notebook]   207   


### Download data asset

Get "credit_risk_training.csv" data asset ID

In [20]:
result = ! cpdctl asset search --project-id {PROJECT_ID} --type-name data_asset --query "asset.name:credit_risk_training.csv" --output json --jmes-query "results[0].metadata.asset_id" --raw-output
DATA_ASSET_ID = result.s
print("'credit_risk_training.csv' data asset ID is: {}".format(DATA_ASSET_ID))

'credit_risk_training.csv' data asset ID is: edb6fe21-77c4-4cb3-aa6d-9e36d2b18edd


Download data asset

In [21]:
! cpdctl asset get --project-id {PROJECT_ID} --asset-id {DATA_ASSET_ID}

...
                  
ID:            edb6fe21-77c4-4cb3-aa6d-9e36d2b18edd   
Name:          credit_risk_training.csv   
Created:       2021-01-29T08:51:00.000Z   
Description:      
Type:          data_asset   
State:         available   
Tags:          []   
Size:          689622   
Attachments:   ID                                     Name                       Type         Mime Type      
               0d7bc498-8913-4a53-91d3-b419e8ba070a   credit_risk_training.csv   data_asset   text/csv      
                  


In [22]:
result = ! cpdctl asset get --project-id {PROJECT_ID} --asset-id {DATA_ASSET_ID} --output json --jmes-query "attachments[0].id" --raw-output
DATA_ATTACHMENT_ID = result.s
print("Data asset attachment ID is: {}".format(DATA_ATTACHMENT_ID))

Data asset attachment ID is: 0d7bc498-8913-4a53-91d3-b419e8ba070a


In [23]:
! cpdctl asset attachment download --project-id {PROJECT_ID} --asset-id {DATA_ASSET_ID} --attachment-id {DATA_ATTACHMENT_ID} --output-path credit_risk_training.csv

...
OK
Output written to credit_risk_training.csv


### Upload a new data asset

Clean up the existing "car_rental_training_data.csv" data assets

In [24]:
result = ! cpdctl asset search --project-id {PROJECT_ID} --type-name data_asset --query "asset.name:car_rental_training_data.csv" --output json --jmes-query "results[*].metadata.asset_id" --raw-output
DATA_ASSET_IDS = json.loads(result.s)
for data_asset_id in DATA_ASSET_IDS:
    print("Deleteing data asset with ID: {}".format(data_asset_id))
    ! cpdctl asset delete --project-id {PROJECT_ID} --asset-id {data_asset_id}

Download the another training data set from github 

In [25]:
! curl https://raw.githubusercontent.com/pmservice/wml-sample-models/master/spark/cars-4-you/data/car_rental_training_data.csv -o car_rental_training_data.csv
! wc -l car_rental_training_data.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 79518  100 79518    0     0   207k      0 --:--:-- --:--:-- --:--:--  207k
     486 car_rental_training_data.csv


Create a new data asset in the project from the downloaded file "credit_risk_training.csv"

In [26]:
! cpdctl asset data-asset upload --file car_rental_training_data.csv --project-id {PROJECT_ID} --progress true --tag "cpdctl-demo" --mime "text/csv"

...
77.87 KiB / 77.65 KiB [---------------------------------------] 100.28% ? p/s 0s
                  
ID:            8a8c8daa-f6eb-4e2b-9526-b6f13a457785   
Name:          car_rental_training_data.csv   
Created:       2021-01-29T08:54:47.000Z   
Description:      
Type:          data_asset   
State:         available   
Tags:          [cpdctl-demo]   
Size:          79518   
Attachments:   ID                                     Name                           Type         Mime Type      
               8c197d6a-ae23-4d30-a8be-0c4ad68e1be3   car_rental_training_data.csv   data_asset   text/csv      
                  


In [27]:
result = ! cpdctl asset search --project-id {PROJECT_ID} --type-name data_asset --query "asset.name:car_rental_training_data.csv" --output json --jmes-query "results[0].metadata.asset_id" --raw-output
NEW_DATA_ASSET_ID = result.s
print("'credit_risk_training.csv' data asset ID is: {}".format(NEW_DATA_ASSET_ID))

'credit_risk_training.csv' data asset ID is: 8a8c8daa-f6eb-4e2b-9526-b6f13a457785


In [28]:
! cpdctl asset search --project-id {PROJECT_ID} --type-name data_asset --query "*:*"

...
ID                                     Name                           Created                    Description   Type         State       Tags            Size   
edb6fe21-77c4-4cb3-aa6d-9e36d2b18edd   credit_risk_training.csv       2021-01-29T08:51:00.000Z                 data_asset   available   []              689622   
8a8c8daa-f6eb-4e2b-9526-b6f13a457785   car_rental_training_data.csv   2021-01-29T08:54:47.000Z                 data_asset   available   [cpdctl-demo]   79518   


Promote the new data asset to "cpdctl-demo-space" space

In [29]:
! cpdctl space list

...
Nothing to show.


Create a new space

In [30]:
! cpdctl space create --name cpdctl-demo-space

...
              
ID:        d9bfa660-1be7-46ab-aa53-e9010a634bba   
Name:      cpdctl-demo-space   
Created:   2021-01-29T08:56:07.389Z   
State:     active   
Tags:      []   


In [31]:
! cpdctl space list

...
ID                                     Name                Created                    State    Tags   
d9bfa660-1be7-46ab-aa53-e9010a634bba   cpdctl-demo-space   2021-01-29T08:56:07.389Z   active   []   


Get the first space ID

In [32]:
result = ! cpdctl space list --output json --jmes-query "resources[0].metadata.id" --raw-output
DEV_SPACE_ID = result.s
print("Space ID is: {}".format(DEV_SPACE_ID))

Space ID is: d9bfa660-1be7-46ab-aa53-e9010a634bba


Clean up existing data asset

In [33]:
result = ! cpdctl asset search --space-id {DEV_SPACE_ID} --type-name data_asset --query "asset.name:car_rental_training_data.csv" --output json --jmes-query "results[*].metadata.asset_id" --raw-output
DATA_ASSET_IDS = json.loads(result.s)
for data_asset_id in DATA_ASSET_IDS:
    print("Deleteing data asset with ID: {}".format(data_asset_id))
    ! cpdctl asset delete --space-id {DEV_SPACE_ID} --asset-id {data_asset_id}

In [34]:
import json

PROMOTE_BODY = {
    "mode": 0,
    "space_id": DEV_SPACE_ID,
    "metadata": {
        "tags": ["cpdctl-demo", "promoted-{}]".format(PROJECT_ID[0:8])]
    }
}
PROMOTE_BODY_JSON = json.dumps(PROMOTE_BODY)

! cpdctl asset promote --project-id {PROJECT_ID} --asset-id {NEW_DATA_ASSET_ID} --request-body '{PROMOTE_BODY_JSON}'

...
OK


In [35]:
! cpdctl asset search --space-id {DEV_SPACE_ID} --type-name asset --query "*:*"

...
ID                                     Name                           Created                    Description   Type         State       Tags                               Size   
783d8fc5-ae2c-47d0-a311-f8890dfa1ce0   car_rental_training_data.csv   2021-01-29T08:57:12.000Z                 data_asset   available   [cpdctl-demo promoted-7fb76cf7]]   79518   


### Train and store model in the notebook

List jobs in the project

In [47]:
result = ! cpdctl asset search --project-id {PROJECT_ID} --type-name job --output json --query "asset.name:train-scikit-model-job" --jmes-query "results[0].metadata.asset_id" --raw-output
EXISTING_JOB_ID = result.s
if EXISTING_JOB_ID and EXISTING_JOB_ID != "null":
    print("Deleteing job with ID: {}".format(EXISTING_JOB_ID))
    ! cpdctl job delete --project-id {PROJECT_ID} --job-id {EXISTING_JOB_ID}

In [41]:
! cpdctl asset search --project-id {PROJECT_ID} --type-name notebook --query "*:*"

...
ID                                     Name                                                Created                    Description                 Type       State       Tags         Size   
ceea9923-7ff7-4084-a560-818716e65b4d   Sample notebook                                     2021-01-29T08:51:49.000Z   Notebook run using cpdctl   notebook   available   [notebook]   207   
e4f0d078-04b1-49a4-b360-2939a2dfab34   train-scikit-model-to-predict-boston-house-prices   2021-01-29T09:06:41.000Z                               notebook   available   [notebook]   19655   


In [43]:
result = ! cpdctl asset search --project-id {PROJECT_ID} --type-name notebook --output json --query "asset.name:train-scikit-model-to-predict-boston-house-prices" --jmes-query "results[0].metadata.asset_id" --raw-output
NOTEBOOK_ID = result.s
print("'train-scikit-model-to-predict-boston-house-prices' notebook ID is: {}".format(NOTEBOOK_ID))

'train-scikit-model-to-predict-boston-house-prices' notebook ID is: e4f0d078-04b1-49a4-b360-2939a2dfab34


In [44]:
! cpdctl environment list --project-id {PROJECT_ID}

...
ID                                                Name         Description   
jupconda37-7fb76cf7-25be-435d-818e-bd6e9b5254f5   jupconda37      
jupconda36-7fb76cf7-25be-435d-818e-bd6e9b5254f5   jupconda36   Default Python 3.6   


In [48]:
JOB = {
    "asset_ref": NOTEBOOK_ID,
    "name": "train-scikit-model-job",
    "configuration": {
        "env_id": "jupconda37-7fb76cf7-25be-435d-818e-bd6e9b5254f5",
        "env_variables": [
            "URL={}".format(CPD35_URL),
            "USER_NAME={}".format(CPD35_USER_NAME),
            "USER_PASSWORD={}".format(CPD35_USER_PASSWORD),
            "SPACE_ID={}".format(DEV_SPACE_ID)
        ]
    }
}
JOB_JSON = json.dumps(JOB)
result = ! cpdctl job create --project-id {PROJECT_ID} --job '{JOB_JSON}' --output json --jmes-query "metadata.asset_id" --raw-output
JOB_ID = result.s
print("'train-scikit-model-job' job ID is: {}".format(JOB_ID))

'train-scikit-model-job' job ID is: aa508183-945b-4c68-82b7-5557ab33a2be


Trigger job run

In [49]:
JOB_RUN = {
    "job_run": {}
}
JOB_RUN_JSON = json.dumps(JOB_RUN)
result = ! cpdctl job run create --project-id {PROJECT_ID} --job-id {JOB_ID} --job-run '{JOB_RUN_JSON}' --async --output json --jmes-query 'metadata.asset_id'
RUN_ID = result.s
print("The new run ID for 'train-scikit-model-job' job: {}".format(RUN_ID))

The new run ID for 'train-scikit-model-job' job: "ec805c4c-1ba9-4a80-82e0-4c05b67043d9"


Get notebook run status

In [53]:
! cpdctl job run get --project-id {PROJECT_ID} --job-id {JOB_ID} --run-id {RUN_ID} --output json --jmes-query "entity.job_run.state" --raw-output

Completed


In [54]:
! cpdctl job run logs --project-id {PROJECT_ID} --job-id {JOB_ID} --run-id {RUN_ID}

...
total_count   results   
180           Cell 8:   
   6;1m180           Collecting ibm-watson-machine-learning
180              
   6;1m180             Downloading ibm_watson_machine_learning-1.0.45-py3-none-any.whl (1.7 MB)
     |▏                               | 10 kB 11.0 MB/s eta 0:00:01   
     |█▍                              | 71 kB 1.4 MB/s eta 0:00:02   
     |████▏                           | 215 kB 1.6 MB/s eta 0:00:01   
     |██████████▎                     | 532 kB 1.6 MB/s eta 0:00:01   
     |████████████████████▉           | 1.1 MB 1.6 MB/s eta 0:00:01   
     |████████████████████████████████| 1.7 MB 1.6 MB/s eta 0:00:01   
        |████████████████████████████████| 1.7 MB 1.6 MB/s 
180              
   6;1m180           Requirement already satisfied, skipping upgrade: urllib3 in /opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages (from ibm-watson-machine-learning) (1.25.9)
   6;1m180           Requirement already satisfied, skipping upgrade: certifi in /op

In [61]:
! cpdctl ml model list --space-id {DEV_SPACE_ID}

...
ID                                     Name                                   Created                    Type                Tags   
384ddcde-a7fe-4652-baa9-b41c40b08284   boston-house-prices-prediction-model   2021-01-29T09:27:11.002Z   scikit-learn_0.23   []   


In [59]:
result = ! cpdctl asset search --space-id {DEV_SPACE_ID} --type-name wml_model --query "asset.name:boston-house-prices-prediction-model" --output json --jmes-query "results[0].metadata.asset_id" --raw-output
MODEL_ID = result.s
print("'boston-house-prices-prediction-model' model ID is: {}".format(MODEL_ID))

'boston-house-prices-prediction-model' model ID is: 384ddcde-a7fe-4652-baa9-b41c40b08284


Export space assets

In [62]:
EXPORT = {
    'asset_ids': [MODEL_ID]
}
EXPORT_JSON = json.dumps(EXPORT)
result = ! cpdctl asset export start --space-id {DEV_SPACE_ID} --assets '{EXPORT_JSON}' --name dev-space-assets --output json --jmes-query "metadata.id"
EXPORT_ID = result.s
print("The new export with ID: {}".format(EXPORT_ID))

The new export with ID: "d66b9fd0-5e3b-4f1a-9e61-ffab16076368"


In [63]:
! cpdctl asset export get --space-id {DEV_SPACE_ID} --export-id {EXPORT_ID}

...
              
ID:        d66b9fd0-5e3b-4f1a-9e61-ffab16076368   
Name:      dev-space-assets   
Created:   2021-01-29T09:30:37.926Z   
State:     completed   


In [64]:
! cpdctl asset export download --space-id {DEV_SPACE_ID} --export-id {EXPORT_ID} --output-file dev-space-assets.zip

...
OK
Output written to dev-space-assets.zip


In [65]:
! unzip -l dev-space-assets.zip 

Archive:  dev-space-assets.zip
  Length      Date    Time    Name
---------  ---------- -----   ----
       44  01-29-2021 09:30   deflate.log
      416  01-29-2021 09:30   assettypes/folder_asset.json
      988  01-29-2021 09:30   assettypes/column_info.json
      234  01-29-2021 09:30   assettypes/policy_transform.json
      543  01-29-2021 09:30   assettypes/asset_terms.json
      459  01-29-2021 09:30   assettypes/omrs_relationship_message.json
      288  01-29-2021 09:30   assettypes/package_extension.json
      465  01-29-2021 09:30   assettypes/connection_credentials.json
      526  01-29-2021 09:30   assettypes/environment.json
      311  01-29-2021 09:30   assettypes/shiny_asset.json
     1442  01-29-2021 09:30   assettypes/job_run.json
    38778  01-29-2021 09:30   assettypes/wml_model.json
     2528  01-29-2021 09:30   assettypes/wml_remote_training_system.json
    27504  01-29-2021 09:30   assettypes/wml_training_definition.json
      327  01-29-2021 09:30   assettypes/omrs

### Create a new QA space and import assets

Ensure there is no QA space

In [71]:
result = ! cpdctl space list --output json --jmes-query "resources[?name == \"cpdctl-demo-qa-space\"] | metadata.id"
SPACE_IDS = json.loads(result.s)
if SPACE_IDS:
    for space_id in SPACE_IDS:
        result = ! cpdctl ml deployment list --space-id {space_id} --output json --jmes-query "resources[*].metadata.id"
        DEPLOYMENT_IDS = json.loads(result.s)
        if DEPLOYMENT_IDS:
            for deployment_id in DEPLOYMENT_IDS:
                print('Deleting deployment with ID: {}'.format(space_id))
                ! cpdctl ml deployment delete --space-id {space_id} --deployment-id {deployment_id}
        print('Deleting space with ID: {}'.format(space_id))
        ! cpdctl space delete --space-id {space_id}

Create a new QA space

In [74]:
result = ! cpdctl space create --name cpdctl-demo-qa-space --output json --jmes-query "metadata.id" --raw-output
QA_SPACE_ID = result.s
print("The new 'cpdctl-demo-qa-space' space ID is: {}".format(QA_SPACE_ID))

The new 'cpdctl-demo-qa-space' space ID is: c280acdb-69ab-4fc3-81cb-86b28f5db80f


Import assets exported from DEV space

In [75]:
result = ! cpdctl asset import start --space-id {QA_SPACE_ID} --import-file dev-space-assets.zip --output json --jmes-query "metadata.id" --raw-output
QA_IMPORT_ID = result.s
print("The new import ID is: {}".format(QA_IMPORT_ID))

The new import ID is: d6b729fa-1b8c-4db0-8e3c-f58cb4746882


In [76]:
! cpdctl asset import get --space-id {QA_SPACE_ID} --import-id {QA_IMPORT_ID}

...
              
ID:        d6b729fa-1b8c-4db0-8e3c-f58cb4746882   
Created:   2021-01-29T09:40:51.099Z   
State:     completed   


In [77]:
! cpdctl ml model list --space-id {QA_SPACE_ID}

...
ID                                     Name                                   Created                    Type                Tags   
2da9a4d6-85c3-458a-9b84-557d1015e955   boston-house-prices-prediction-model   2021-01-29T09:41:05.002Z   scikit-learn_0.23   []   


In [79]:
result = ! cpdctl ml model list --space-id {QA_SPACE_ID} --output json --jmes-query "resources[0].metadata.id" --raw-output
QA_MODEL_ID = result.s
print("QA model ID is: {}".format(QA_MODEL_ID))

QA model ID is: 2da9a4d6-85c3-458a-9b84-557d1015e955


In [80]:
ASSET_JSON = json.dumps({"id": QA_MODEL_ID})
ONLINE_JSON = json.dumps({})

! cpdctl ml deployment create --space-id {QA_SPACE_ID} --asset '{ASSET_JSON}' --online '{ONLINE_JSON}'

...
              
ID:        cc9b9665-dfc6-4604-aae1-efd75a97b911   
Name:         
Created:   2021-01-29T09:41:49.154Z   
State:     ready   
Tags:      []   


In [81]:
! cpdctl ml deployment list  --space-id {QA_SPACE_ID}

...
ID                                     Name   Created                    State   Tags   
cc9b9665-dfc6-4604-aae1-efd75a97b911          2021-01-29T09:41:49.154Z   ready   []   


### Author

Rafał Bigaj, System Architect with long successful record of building and leading teams. Broad and practical knowledge in the area of cloud computing, machine learning and distributed systems development. 

Copyright © 2020 IBM. This notebook and its source code are released under the terms of the MIT License.